<a href="https://colab.research.google.com/github/Kuzay3t/2D-Image-Reconstruction/blob/main/2d_image_reconstruction_with_CFair10_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recontructing C-FAIR 10 WITH PYTORCH

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [27]:
# loading the dataset
transform = transforms.Compose([transforms.ToTensor()])

In [28]:
# loading the dataset
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [29]:

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)


In [30]:
# adding noise to the image

def add_noise(images, noise_factor=0.2):
    noisy_images = images + noise_factor * torch.randn_like(images)
    noisy_images = torch.clip(noisy_images, 0., 1.)
    return noisy_images



In [31]:
# building the autoencoder model

class CIFARAutoencoder(nn.Module):
    def __init__(self):
        super(CIFARAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),  # (32x16x16)
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # (64x8x8)
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),# (128x4x4)
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), # (64x8x8)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),  # (32x16x16)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1, output_padding=1),   # (3x32x32)
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [32]:
# setting the model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CIFARAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# training the model

num_epochs = 10
for epoch in range(num_epochs):
    for images, _ in train_loader:
        images = images.to(device)
        noisy_images = add_noise(images).to(device)

        outputs = model(noisy_images)
        loss = criterion(outputs, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.0081
Epoch [2/10], Loss: 0.0063
Epoch [3/10], Loss: 0.0055
Epoch [4/10], Loss: 0.0050
Epoch [5/10], Loss: 0.0050
Epoch [6/10], Loss: 0.0044


In [ ]:
# visualising the reconstruction

model.eval()
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        noisy_images = add_noise(images).to(device)
        reconstructed = model(noisy_images)
        mse_score = F.mse_loss(reconstructed, images) # Calculate MSE here
        break  # single batch

In [ ]:
# Plot 6 examples
fig, axes = plt.subplots(3, 6, figsize=(12, 6))
for i in range(6):
    axes[0, i].imshow(images[i].permute(1, 2, 0).cpu().numpy())
    axes[0, i].set_title("Original")
    axes[0, i].axis("off")

    axes[1, i].imshow(noisy_images[i].permute(1, 2, 0).cpu().numpy())
    axes[1, i].set_title("Noisy")
    axes[1, i].axis("off")

    axes[2, i].imshow(reconstructed[i].permute(1, 2, 0).cpu().numpy())
    axes[2, i].set_title("Denoised")
    axes[2, i].axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# measuring the reconstruction performance

# mse_score = F.mse_loss(reconstructed, images) # This line is moved
print("CIFAR-10 PyTorch Reconstruction MSE:", mse_score.item()) # Now mse_score is defined